In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-12-09 06:20:02--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.56MB/s    in 0.2s    

2021-12-09 06:20:03 (5.56 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BidData-HW-Level1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in csv from S3 into a DataFrame
url = "https://toys-review-bucket.s3.amazonaws.com/amazon_reviews_us_Kitchen_v1_00.tsv"
spark.sparkContext.addFile(url)

In [5]:
df=spark.read.csv(SparkFiles.get("amazon_reviews_us_Kitchen_v1_00.tsv"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

In [6]:
# print(df.count())
df = df.dropna()
print(df.count())
# df = df.dropDuplicates()
# print(df.count())

4879961


In [7]:
from pyspark.sql.functions import to_date

id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date","yyyy-MM-dd").alias("review_date")])
id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3DT59XH7HXR9K|   37000337|B00303FI0G|     529320574| 2015-08-31|
|R1LFS11BNASSU8|   15272914|B00JCZKZN6|     274237558| 2015-08-31|
|R296RT05AG0AF6|   36137863|B00JLIKA5C|     544675303| 2015-08-31|
|R3V37XDZ7ZCI3L|   43311049|B000GBNB8G|     491599489| 2015-08-31|
|R14GU232NQFYX2|   13763148|B00VJ5KX9S|     353790155| 2015-08-31|
| RZQH4V7L2O1PL|   19009420|B00HYB5YY0|     432241873| 2015-08-31|
|R1F8JMOSPJ3KO7|   40599388|B000HEBAV2|     584680984| 2015-08-31|
|R1ZISGY2BWW4Z5|   22719359|B0012DS4GG|     772637306| 2015-08-31|
|R17PW4I3AE5WZW|   47478640|B00FLQ4EE6|     755416578| 2015-08-31|
|R3D93G1KTP6A8P|   34195504|B00DBS9OTG|     648762742| 2015-08-31|
|R18TQIW1NKPUNU|   19100570|B00AN9UJ68|     495720940| 2015-08-31|
|R34KUNL21WU248|   10299811|B00L2P0KNO|      41330497| 2015-08

In [8]:
product_df = df.select(["product_id", "product_title"]).drop_duplicates()
product_df.show(20)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B004NWK4W0|Wildkin Pink Leop...|
|B00IE70YFC|Wilton 415-2178 3...|
|B00004W499|George Foreman GG...|
|B000Q945FG|BIA Cordon Bleu B...|
|B007WTOIPA|Clorox Sales Co B...|
|B006IC09P0|Utenlid Acrylic R...|
|B00CDSU2MI|2000 Diamond Tabl...|
|B000WJMTNA|OXO Good Grips St...|
|B00DYRI13O|Reed & Barton 148...|
|B005FPD67Y|     Ergo Chef Tongs|
|B00JLA6POU|TeChef - Art Pan ...|
|B00FN3P7X4|Hamilton Beach 1....|
|B000UVHUCU|KitchenAid Ultra ...|
|B00IZCWYR2|PackIt Freezable ...|
|B0105V7VS2|Top Rated Lemon S...|
|B002P667TU|Browne (22126P) 6...|
|B00KIYWDTO|Wolfgang Puck Pre...|
|B00004S9EJ|Cuisinart DLC-8SY...|
|B00D19SSE2|Ozeri Serafino Do...|
|B00JV8PTYY|Thermos 12 Ounce ...|
+----------+--------------------+
only showing top 20 rows



In [9]:
reviews_df = df.select(["review_id", "review_headline", "review_body"])
reviews_df.show(15)

+--------------+--------------------+--------------------+
|     review_id|     review_headline|         review_body|
+--------------+--------------------+--------------------+
|R3DT59XH7HXR9K|Beautiful. Looks ...|Beautiful.  Looks...|
|R1LFS11BNASSU8| Awesome & Self-ness|I personally have...|
|R296RT05AG0AF6|Fabulous and wort...|Fabulous and wort...|
|R3V37XDZ7ZCI3L|          Five Stars|A must if you lov...|
|R14GU232NQFYX2|     Better than sex|Worth every penny...|
| RZQH4V7L2O1PL|Does not work on ...|The description s...|
|R1F8JMOSPJ3KO7|Awesome! First fr...|Awesome! First fr...|
|R1ZISGY2BWW4Z5|          Five Stars|Very good item. Q...|
|R17PW4I3AE5WZW|          Five Stars|sharp and look great|
|R3D93G1KTP6A8P|         Three Stars|Should have come ...|
|R18TQIW1NKPUNU|          Five Stars|  my friend loves it|
|R34KUNL21WU248|          Four Stars|works as expected...|
|R2YA1ZA53X12IN|               great|               great|
|R2ZD1IGC9UU55X|         Great Pots!|Fantastic product..

In [20]:
customer_df =df.groupBy("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)",  "customer_count")
customer_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   23042837|             1|
|   48875707|             2|
|   48088660|             1|
|   20781887|             6|
|    1075003|             1|
|   28377689|             3|
|    8338749|             1|
|   39271457|             6|
|   38209321|             5|
|   51397605|             1|
+-----------+--------------+
only showing top 10 rows



In [11]:
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3DT59XH7HXR9K|          5|            0|          0|   N|
|R1LFS11BNASSU8|          5|            0|          1|   N|
|R296RT05AG0AF6|          5|            0|          0|   N|
|R3V37XDZ7ZCI3L|          5|            0|          1|   N|
|R14GU232NQFYX2|          5|            0|          0|   N|
| RZQH4V7L2O1PL|          1|            1|          1|   N|
|R1F8JMOSPJ3KO7|          5|            0|          0|   N|
|R1ZISGY2BWW4Z5|          5|            0|          0|   N|
|R17PW4I3AE5WZW|          5|            0|          0|   N|
|R3D93G1KTP6A8P|          3|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [ ]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [12]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://database-1.cnt6k9yb90vf.us-west-1.rds.amazonaws.com:5432/kitchen_reviews"
config = {"user":"postgres",
          "password": "postgres",
          "driver":"org.postgresql.Driver"}

In [19]:
# Write DataFrame to table

id_df.write.jdbc(url=jdbc_url, table='id_df', mode=mode, properties=config)

In [17]:
product_df.write.jdbc(url=jdbc_url, table='product_id', mode=mode, properties=config)


In [15]:
reviews_df.write.jdbc(url=jdbc_url, table='reviews', mode=mode, properties=config)

In [21]:
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [13]:
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [ ]:
#Create each table in sql

CREATE TABLE id_df (
	review_id VARCHAR,
	customer_id VARCHAR,
	product_id VARCHAR,
	product_parent VARCHAR,
	review_date DATE
);
	
CREATE TABLE product_id(
	product_id VARCHAR,
	product_title VARCHAR
);

CREATE TABLE reviews (
	review_id VARCHAR,
	review_headline VARCHAR,
	review_body VARCHAR
);

CREATE TABLE customers (
	customer_id VARCHAR,
	customer_count INT
);

CREATE TABLE vine_table (
	review_id VARCHAR,
	star_rating INT,
	helpful_votes INT,
  total_votes INT,
  vine VARCHAR
);